In [27]:
#**************RSA_Encryption******************************
import rsa
from rsa.pkcs1 import encrypt, decrypt

# generate public and private keys with rsa.newkeys method,this method accepts key length(atleast 16) as its parameter
publicKey, privateKey = rsa.key.newkeys(512)

# this is the string that we will be encrypting
file = open("message.txt","r")
mess = file.read()
file.close()

# rsa.encrypt method is used to encrypt string with public key string should be encode to byte string before encryption
encMessage = encrypt(mess.encode(),publicKey)

print("original string: ", mess)
print("encrypted string: ", encMessage)

file = open("rsa_encrypted.txt","wb")
file.write(encMessage)
file.close()

original string:  Information security management
encrypted string:  b"S\xcc\xd0sld\x12B\xdd#Xe\xf5+\xf3\x94\x0c\xa9\xb77]\xd5\xfd\xfbi\x9a\x9es\x94\xb6y\x8c\x12\xa8\x1e!\x82'\xdc\x98\x0f\xfd\xee\xbf\xc7~8p[\xeac8\x9d\xb5\x04<\x8b,)2|\x04\x81x"


In [28]:
#***************Base64_+_Binary_Conversion*************************
import base64
#, encoding="unicode_escape"
with open("rsa_encrypted.txt", "rb") as File:
 str1= base64.b64encode(File.read())

filename = 'base64_encrypt.txt'
# we are considering a file to store the string.
with open(filename, 'wb') as f:
 f.write(str1)

file = open("base64_encrypt.txt","r")
text=file.read()
file.close()

# using join() + ord() + format() to convert into binary
bin_result = ''.join(format(ord(x), '08b') for x in text)

file = open("Binary.txt","w")
file.write(bin_result)
file.close()
print("binary string: ", bin_result)

binary string:  01010101001110000111101001010001011000110011001001111000011010110100010101101011010011000110010001001001001100010110100001101100001110010101001101110110011110100110110001000001011110010111000001110100011110100110010001100100001100010110011000110011001101110110000101011010011100010110010101100011001101010101001100110010011001010101100101110111010100110111000101000010001101000110100001100111011010010110011001100011011011010100000100101111001110010011011101110010001011110100100001100110011010100110100001110111010101110010101101110000011010100100111101001010001100100011000101000010010001000111100101001100010011000100001101101011011110010110011001000001010100110100001001100101010000010011110100111101


In [29]:
#**************************Steganography_Encryption*************************
# PIL module is used to extract pixels of image and modify it
from PIL import Image

# Convert encoding data into 8-bit binary form using ASCII value of characters
def genData(data):

  # list of binary codes of given data
  newd = []

  for i in data:
   newd.append(format(ord(i), '08b'))
  return newd

# Pixels are modified according to the 8-bit binary data and finally returned
def modPix(pix, data):

 datalist = genData(data)
 lendata = len(datalist)
 imdata = iter(pix)

 for i in range(lendata):

  # Extracting 3 pixels at a time
  pix = [value for value in imdata.__next__()[:3] + imdata.__next__()[:3] + imdata.__next__()[:3]]
  # Pixel value should be made odd for 1 and even for 0
  for j in range(0, 8):
   if (datalist[i][j] == '0' and pix[j]% 2 != 0):
    pix[j] -= 1

   elif (datalist[i][j] == '1' and pix[j] % 2 == 0):
    if(pix[j] != 0):
     pix[j] -= 1
    else:
     pix[j] += 1

  # Eighth pixel of every set tells whether to stop ot read further.
  # 0 means keep reading; 1 means the message is over.
  if (i == lendata - 1):
   if (pix[-1] % 2 == 0):
    if(pix[-1] != 0):
     pix[-1] -= 1
    else:
     pix[-1] += 1
  else:
   if (pix[-1] % 2 != 0):
    pix[-1] -= 1

  pix = tuple(pix)
  yield pix[0:3]
  yield pix[3:6]
  yield pix[6:9]

def encode_enc(newimg, data):
 w = newimg.size[0]
 (x, y) = (0, 0)

 for pixel in modPix(newimg.getdata(), data):

  # Putting modified pixels in the new image
  newimg.putpixel((x, y), pixel)
  if (x == w - 1):
   x = 0
   y += 1
  else:
   x += 1

# Encode data into image
def encode():
 img = input("Enter image name of the image to be encrypted(with extension) : ")
 image = Image.open(img, 'r')

 file = open("Binary.txt","r")
 data = file.read()
 file.close()
   
 if (len(data) == 0):
  raise ValueError('Data is empty')

 newimg = image.copy()
 encode_enc(newimg, data)

 new_img_name = input("Enter the name of new encrypted image(with extension) : ")
 newimg.save(new_img_name, str(new_img_name.split(".")[1].upper()))

# Decode the data in the image
def decode():
 img = input("Enter the name of the image to be decrypted(with extension) : ")
 image = Image.open(img, 'r')

 data = ''
 imgdata = iter(image.getdata())

 while (True):
  pixels = [value for value in imgdata.__next__()[:3] + imgdata.__next__()[:3] + imgdata.__next__()[:3]]

  # string of binary data
  binstr = ''

  for i in pixels[:8]:
   if (i % 2 == 0):
    binstr += '0'
   else:
    binstr += '1'

  data += chr(int(binstr, 2))
  if (pixels[-1] % 2 != 0):
   return data 

# Main Function
def main():
 file = open("Binary.txt","r")
 bin_data = file.read()
 file.close()
 encode()
 print("Encryption completed")
 dec_data = decode()
 file = open("RSA_encrypte.txt","w")
 file.write(dec_data)
 file.close()
 print("Image decrpyted")
 print("Binary string from decrypted image: ", dec_data)


# Calling main function
main()


Enter image name of the image to be encrypted(with extension) : image1.png
Enter the name of new encrypted image(with extension) : encrypted_image.png
Encryption completed
Enter the name of the image to be decrypted(with extension) : encrypted_image.png
Image decrpyted
Binary string from decrypted image:  010101010011100001111010010100010110001100110010011110000110101101000101011010110100110001100100010010010011000101101000011011000011100101010011011101100111101001101100010000010111100101110000011101000111101001100100011001000011000101100110001100110011011101100001010110100111000101100101011000110011010101010011001100100110010101011001011101110101001101110001010000100011010001101000011001110110100101100110011000110110110101000001001011110011100100110111011100100010111101001000011001100110101001101000011101110101011100101011011100000110101001001111010010100011001000110001010000100100010001111001010011000100110001000011011010110111100101100110010000010101001101000010011001010100000100111

In [30]:
#*************************Binary_to_String_+_Base64***********************************
# Defining BinarytoDecimal() function
def BinaryToDecimal(binary):
 binary1 = binary
 decimal, i, n = 0, 0, 0
 while(binary != 0):
  dec = int(binary) % 10
  decimal = decimal + dec * pow(2, i)
  binary = int(binary)//10
  i += 1
 return (decimal)

file = open("RSA_encrypte.txt","r")
bin_data=file.read()
file.close()

# initializing a empty string for
# storing the string data
str_data =''

# slicing the input and converting it
# in decimal and then converting it in string
for i in range(0, len(bin_data), 8):
 # slicing the bin_data from index range [0, 8] and storing it in temp_data
 temp_data = bin_data[i:i + 8]
 # passing temp_data in BinarytoDecimal() function
 # to get decimal value of corresponding temp_data
 decimal_data = BinaryToDecimal(temp_data)
 # in str_data
 str_data = str_data + chr(decimal_data)

# printing the result
file = open("String.txt","w")
file.write(str_data)
file.close()

import base64
with open("String.txt", "rb") as File:
 str2= (File.read())
textdata = base64.b64decode(str2)
filename = 'Decrypted_message.txt'
with open(filename, 'wb') as f:
 f.write(textdata)
print("decrypted string: ", textdata)
#*********************Decryption_Done*****************************************

decrypted string:  b"S\xcc\xd0sld\x12B\xdd#Xe\xf5+\xf3\x94\x0c\xa9\xb77]\xd5\xfd\xfbi\x9a\x9es\x94\xb6y\x8c\x12\xa8\x1e!\x82'\xdc\x98\x0f\xfd\xee\xbf\xc7~8p[\xeac8\x9d\xb5\x04<\x8b,)2|\x04\x81x"


In [33]:
#*****************************Rsa_Decryption*****************************************
# this is the string that we will be encrypting
file = open("Decrypted_message.txt","rb")
message = file.read()
file.close()
# the encrypted message can be decrypted with ras.decrypt method and private key
# decrypt method returns encoded byte string, use decode method to convert it to string
# public key cannot be used for decryption
decMessage = decrypt(message, privateKey).decode()
print("decrypted message: ", decMessage)
file = open("Rsa_decrypted.txt","w")
file.write(decMessage)
file.close()
print('Decryption Done')

decrypted message:  Information security management
Decryption Done
